## Data Inspection and Cleaning for Train and Test Datasets

In this section the train and test datasets are inspected and cleaned. They are then merged and saved for further exploratory data analysis (EDA) and modelling.

In [1]:
# Import libraries

import numpy as np 
import pandas as pd 
import time
import requests
import random
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# This library is to allow file handling between mac and win without errors

from pathlib import Path

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)



/Users/Dickson/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Import datasets

data_folder = Path("data/")
df_train= pd.read_csv(data_folder / "train.csv")
df_test= pd.read_csv(data_folder / "test.csv")
df_weather = pd.read_csv(data_folder / "weather.csv")
df_spray =pd.read_csv(data_folder / "spray.csv")

In [3]:
# Inspect the datasets

df_train.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [4]:
# Change the column titles to lower case.
# Compare column names to see which are common and which are not. 

df_train.columns = map(str.lower, df_train.columns)
df_test.columns = map(str.lower, df_test.columns)

print(df_train.columns.intersection(df_test.columns))
print("----------------------------------------")
print(df_train.columns.difference(df_test.columns))

Index(['date', 'address', 'species', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy'],
      dtype='object')
----------------------------------------
Index(['nummosquitos', 'wnvpresent'], dtype='object')


In [5]:
# wnvpresent is the key class we are interested in.

# The class is imbalanced.

df_train['wnvpresent'].value_counts()

0    9955
1     551
Name: wnvpresent, dtype: int64

In [6]:
# Check the number of columns and rows

df_test.shape

(116293, 11)

In [7]:
# Merge the train and test datasets

df_full = pd.concat([df_train, df_test])

In [8]:
# Check for missing values in the merged dataset

missing_stats = []

for col in df_full.columns:
    missing_stats.append((col, df_full[col].nunique(), df_full[col].isnull().sum() * 100 / df_full.shape[0], df_full[col].value_counts(normalize=True, dropna=False).values[0] * 100, df_full[col].dtype))
    
stats_df = pd.DataFrame(missing_stats, columns=['Feature', 'Unique_values', 'Percentage of missing values', 'Percentage of values in the biggest category', 'type'])
stats_df.sort_values('Percentage of missing values', ascending=False)

,Feature,Unique_values,Percentage of missing values,Percentage of values in the biggest category,type
10,nummosquitos,50,91.714446,91.714446,float64
11,wnvpresent,2,91.714446,91.714446,float64
12,id,116293,8.285554,8.285554,float64
0,date,190,0.000000,1.019724,object
1,address,151,0.000000,1.749225,object
2,species,8,0.000000,15.860535,object
3,block,65,0.000000,7.970094,int64
4,street,139,0.000000,1.749225,object
5,trap,149,0.000000,1.749225,object
6,addressnumberandstreet,151,0.000000,1.749225,object


In [9]:
# We want to assign the 2 weather stations's data to the train and test datasets.

# Weather data are measured daily for each station. 
# we use road number 290, a mid point between station 1 & 2 to infer which station the train & test sets data belong to
# the lat of rd 290 is 41.876378
# after which, we input weather info to the train & test data based on their infered station and based on the data of the data point

# higher = station 1
# lower =  station 2


df_full['station'] =[1 if x >41.876378 else 2 for x in df_full['latitude']]

In [10]:
df_full[['station','latitude']].head(100)

,station,latitude
0,1,41.954690
1,1,41.954690
2,1,41.994991
3,1,41.974089
4,1,41.974089
5,1,41.921600
6,1,41.891118
7,2,41.867108
8,2,41.867108
9,1,41.896282


In [11]:
df_full.shape

(126799, 14)

In [12]:
# Check the number of stations in the merged dataset, which should correspond to a total of 126799.

df_full['station'].value_counts()

2    69168
1    57631
Name: station, dtype: int64

In [13]:
# Save the merged dataset

df_full.to_csv('./data/traintest.csv', index=False)